<a href="https://colab.research.google.com/github/4k5h1t/Network-Anomaly-Detection/blob/main/Network_Anomaly_Detection_Model_(Traditional).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d solarmainframe/ids-intrusion-csv

100% 1.60G/1.60G [01:06<00:00, 29.7MB/s]
100% 1.60G/1.60G [01:06<00:00, 25.8MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

Archive:  ids-intrusion-csv.zip
replace 02-14-2018.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: 02-15-2018.csv          
  inflating: 02-16-2018.csv          
  inflating: 02-20-2018.csv          
  inflating: 02-21-2018.csv          
  inflating: 02-22-2018.csv          
  inflating: 02-23-2018.csv          
  inflating: 02-28-2018.csv          
  inflating: 03-01-2018.csv          
  inflating: 03-02-2018.csv          


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.utils import class_weight

RANDOM_STATE_SEED = 12

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Kaggle/02-14-2018.csv')
df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,80,6,14/02/2018 10:53:23,10156986,5,5,1089,1923,587,0,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
1048571,80,6,14/02/2018 10:53:33,117,2,0,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
1048572,80,6,14/02/2018 10:53:28,5095331,3,1,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
1048573,80,6,14/02/2018 10:53:28,5235511,3,1,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [ ]:
df = df.drop(['Timestamp'], axis=1) # Reorder this
df.dropna(inplace=True)
df

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,112641719,3,0,0,0,0,0,0.000000,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,112641466,3,0,0,0,0,0,0.000000,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,112638623,3,0,0,0,0,0,0.000000,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,6453966,15,10,1239,2273,744,0,82.600000,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,22,6,8804066,14,11,1143,2209,744,0,81.642857,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,80,6,10156986,5,5,1089,1923,587,0,217.800000,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
1048571,80,6,117,2,0,0,0,0,0,0.000000,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
1048572,80,6,5095331,3,1,0,0,0,0,0.000000,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
1048573,80,6,5235511,3,1,0,0,0,0,0.000000,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [ ]:
df["Label"].value_counts()


Benign            665355
FTP-BruteForce    193354
SSH-Bruteforce    187589
Name: Label, dtype: int64

In [ ]:
df.replace(to_replace=["FTP-BruteForce", "SSH-Bruteforce"], value="Malicious", inplace=True)
df1 = df[df["Label"] == "Benign"][:380943]
df2 = df[df["Label"] == "Malicious"][:380943]
df_equal = pd.concat([df1,df2], axis =0)
df_equal["Label"].value_counts()

Benign       380943
Malicious    380943
Name: Label, dtype: int64

In [ ]:
df = df_equal
df.replace(to_replace="Benign", value=0, inplace=True)
df.replace(to_replace="Malicious", value=1, inplace=True)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [ ]:
X = df.drop(['Label'], axis=1)
Y = df['Label']
X.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 761053 entries, 0 to 383186
Data columns (total 78 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           761053 non-null  int64  
 1   Protocol           761053 non-null  int64  
 2   Flow Duration      761053 non-null  int64  
 3   Tot Fwd Pkts       761053 non-null  int64  
 4   Tot Bwd Pkts       761053 non-null  int64  
 5   TotLen Fwd Pkts    761053 non-null  int64  
 6   TotLen Bwd Pkts    761053 non-null  int64  
 7   Fwd Pkt Len Max    761053 non-null  int64  
 8   Fwd Pkt Len Min    761053 non-null  int64  
 9   Fwd Pkt Len Mean   761053 non-null  float64
 10  Fwd Pkt Len Std    761053 non-null  float64
 11  Bwd Pkt Len Max    761053 non-null  int64  
 12  Bwd Pkt Len Min    761053 non-null  int64  
 13  Bwd Pkt Len Mean   761053 non-null  float64
 14  Bwd Pkt Len Std    761053 non-null  float64
 15  Flow Byts/s        761053 non-null  float64
 16  Fl

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_STATE_SEED, shuffle=True)

In [ ]:
print(Xtrain.shape)
print(Xtest.shape)
print(Ytrain.shape)
print(Ytest.shape)

(532737, 78)
(228316, 78)
(532737,)
(228316,)


In [ ]:
model = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None
)

hyperparameters = {
    'n_estimators': [50, 75, 100, 125, 150]
}

In [ ]:
clf = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    cv=5,
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

In [ ]:
clf.fit(X=Xtrain, y=Ytrain)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'n_estimators': [50, 75, 100, 125, 150]}, verbose=1)

In [ ]:
print("Accuracy score on Validation set: \n")
print(clf.best_score_ )
print("---------------")
print("Best performing hyperparameters on Validation set: ")
print(clf.best_params_)
print("---------------")
print(clf.best_estimator_)

In [ ]:
model = clf.best_estimator_

In [ ]:
model

In [ ]:
predictions = model.predict(Xtest)

In [ ]:
print(accuracy_score(Ytest, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Ytest, predictions)
import seaborn as sns
sns.heatmap(cf_matrix, annot=True)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers

In [ ]:
neuralNetModel = keras.Sequential([
    layers.InputLayer(input_shape=(78,)),
       
    layers.BatchNormalization(renorm=True),
    layers.Dense(128, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.BatchNormalization(renorm=True),
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.BatchNormalization(renorm=True),
    layers.Dense(32, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(1, activation='sigmoid'),
])

neuralNetModel.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

from tensorflow.keras import callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta = 0.001,
    patience = 5,
    restore_best_weights = True
)

neuralNetModel.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = neuralNetModel.fit(
    Xtrain, Ytrain,
    epochs=50,
    batch_size = 256,
    callbacks=[early_stopping]
)

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss']].plot()
history_frame.loc[:, ['binary_accuracy']].plot();

In [ ]:
predictions=(neuralNetModel.predict(Xtest) > 0.5).astype("int32")

In [ ]:
print(accuracy_score(Ytest, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Ytest, predictions)
import seaborn as sns
sns.heatmap(cf_matrix, annot=True)

In [ ]:
def freq_count(data):
    mp = dict();
    for i in data:
        if i in mp:
            mp[i] = mp[i]+1
        else:
            mp[i] = 1
    return mp

Ytrain[Ytrain == 1] = -1
Ytrain[Ytrain == 0] = 1
print(freq_count(Ytrain))

Ytest[Ytest == 1] = -1
Ytest[Ytest == 0] = 1
freq_count(Ytest)

In [ ]:
from sklearn.ensemble import IsolationForest
iFM = IsolationForest(
    n_estimators = 100,
    max_samples = "auto",
    random_state = 42,
    warm_start = False
)

In [ ]:
hyperparameters = {
    'n_estimators': [50, 75, 100, 125, 150]
}

isolationForestCV = GridSearchCV(
    estimator=iFM,
    scoring = 'accuracy',
    param_grid=hyperparameters,
    cv=5,
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

In [ ]:
isolationForestCV.fit(X = Xtrain, y = Ytrain)

In [ ]:
print("Accuracy score on Validation set: \n")
print(isolationForestCV.best_score_ )
print("---------------")
print("Best performing hyperparameters on Validation set: ")
print(isolationForestCV.best_params_)
print("---------------")
print(isolationForestCV.best_estimator_)

In [ ]:
model = isolationForestCV.best_estimator_

In [ ]:
model

In [ ]:
predictions = model.predict(Xtest)
freq_count(predictions)

In [ ]:
print(accuracy_score(Ytest, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Ytest, predictions)
cf_matrix

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix, annot=True)